In [1]:
from elasticsearch import Elasticsearch

In [10]:
## connect to the elasti search with python

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "iiGa0a1_zTfj8ijJPksW"),
    ca_certs= "E:\\DLytica_train\\semantic_search_engine\\elasticsearch-8.15.2\\config\\certs\\http_ca.crt"           
    ## authentication certificate : present in the config directory of elasticsearch-unzip folder
)

es.ping()
##   if ping is "TURE" connection established successfully



True

## Prepare the data

In [26]:
import pandas as pd

df = pd.read_csv("laptop_data.csv")
df.head()

## Here , i am trying to search Semantically the product_title
# so lets convert the prodcut_title into vectors 



,link,product_title,price,actual_price,ratings,color
0,https://www.daraz.com.np/products/dell-vostro-...,Dell Vostro 3520 i3 12th Gen | 16GB RAM | 512G...,"Rs. 55,999",NaN,13 Ratings,Black
1,https://www.daraz.com.np/products/apple-macboo...,Apple MacBook Air 13-inch M1 256GB - Oliz Store,"Rs. 109,900","Rs. 139,900",76 Ratings,Space Grey
2,https://www.daraz.com.np/products/dell-vostro-...,Dell Vostro 3520 i5 12th Gen | 16GB RAM | 512G...,"Rs. 68,999",NaN,22 Ratings,Black
3,https://www.daraz.com.np/products/dell-vostro-...,Dell Vostro 3520 i5 12th Gen | 8GB RAM | 256GB...,"Rs. 64,000",NaN,8 Ratings,Black
4,https://www.daraz.com.np/products/acer-nitro-v...,Acer Nitro V 15 i7 13th Gen 13620H | 16GB DDR5...,"Rs. 137,999",NaN,2 Ratings,Black


In [37]:
df.isna().value_counts()

link   product_title  price  actual_price  ratings  color
False  False          False  True          False    False    364
                             False         False    False    234
                      True   True          False    False      1
Name: count, dtype: int64

In [40]:
df = df.fillna(value = "None")
df.isna().value_counts()

link   product_title  price  actual_price  ratings  color
False  False          False  False         False    False    599
Name: count, dtype: int64

## Converting fileds to Vector using BERT model


In [41]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-mpnet-base-v2")

## Loaded model description

## Max Sequence Length:384
## Dimensions:768
## Normalized Embeddings:true
## Suitable Score Functions:dot-product (util.dot_score), 
#                       cosine-similarity (util.cos_sim), 
#                       euclidean distance

e:\DLytica_train\semantic_search_engine\elastic_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [42]:
# convert the "product_title" field into the embedding vectors using pretraind BERT model 
# after the vectors are converted wriiting them to datafarme with label/id  - "DescriptionVector"

df["DecriptionVector"] = df["product_title"].apply(lambda x: model.encode(x))

In [43]:
df.head()

,link,product_title,price,actual_price,ratings,color,DecriptionVector
0,https://www.daraz.com.np/products/dell-vostro-...,Dell Vostro 3520 i3 12th Gen | 16GB RAM | 512G...,"Rs. 55,999",None,13 Ratings,Black,"[-0.02164589, -0.033171486, -0.0061107734, 0.0..."
1,https://www.daraz.com.np/products/apple-macboo...,Apple MacBook Air 13-inch M1 256GB - Oliz Store,"Rs. 109,900","Rs. 139,900",76 Ratings,Space Grey,"[-0.012230367, -0.009784535, -0.030027973, 0.0..."
2,https://www.daraz.com.np/products/dell-vostro-...,Dell Vostro 3520 i5 12th Gen | 16GB RAM | 512G...,"Rs. 68,999",None,22 Ratings,Black,"[-0.021032594, -0.029408198, -0.007967253, 0.0..."
3,https://www.daraz.com.np/products/dell-vostro-...,Dell Vostro 3520 i5 12th Gen | 8GB RAM | 256GB...,"Rs. 64,000",None,8 Ratings,Black,"[-0.021730883, -0.021214992, -0.007739092, 0.0..."
4,https://www.daraz.com.np/products/acer-nitro-v...,Acer Nitro V 15 i7 13th Gen 13620H | 16GB DDR5...,"Rs. 137,999",None,2 Ratings,Black,"[-0.011995952, -0.0009470535, -0.0058524557, 0..."


In [23]:
es.ping()

True

## **Create new index in the Elastic search**

In [45]:
#to delete the created index in elastic search
#  es.indices.delete(index="laptop_products")

ObjectApiResponse({'acknowledged': True})

In [46]:
### ELASTIC SEARCH (json databaes) - take data input in format of JSON - convert dataframe(tabular) to JSON

from indexMapping import indexMapping

es.indices.create(index="laptop_products", mappings=indexMapping)

# creates the new index in ELASTIC SEARCH --if  ACK : TURE - index created successfully


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'laptop_products'})

## **Ingest the data into index**
    * convert tabular data to JSON
    * ingest the data to elastic search index

In [47]:
record_list = df.to_dict("records")

# converted data to JSON format

In [48]:
record_list[0]

# verifying - data is in key-value pair

{'link': 'https://www.daraz.com.np/products/dell-vostro-3520-i3-12th-gen-16gb-ram-512gb-ssd-156-120hz-fhd-display-laptop-i129017788.html',
 'product_title': 'Dell Vostro 3520 i3 12th Gen | 16GB RAM | 512GB SSD | 15.6" 120Hz FHD Display Laptop',
 'price': 'Rs. 55,999',
 'actual_price': 'None',
 'ratings': '13 Ratings',
 'color': 'Black',
 'DecriptionVector': array([-2.16458905e-02, -3.31714861e-02, -6.11077342e-03,  1.97077338e-02,
         2.02681180e-02, -1.07456660e-02, -1.59642398e-02,  7.43361562e-02,
        -6.68350458e-02, -2.11666524e-03, -4.73406576e-02, -3.79428603e-02,
         2.02940311e-02,  1.04677200e-01, -8.83905683e-03,  2.61962395e-02,
         4.27645259e-02, -7.46356603e-03, -6.44840598e-02, -1.51700722e-02,
        -5.18629849e-02, -3.83023694e-02,  2.85153650e-02, -3.60783115e-02,
        -4.36770171e-02, -2.44487207e-02, -4.49609756e-03,  2.04229094e-02,
        -4.41121534e-02,  2.35450938e-02,  2.87339790e-04,  7.37686362e-03,
         6.10066429e-02,  4.63647

In [49]:
#  indexing/ingesting all the data into the created laptop_products- index

for record in record_list:
    try:
        es.index(index="laptop_products", document=record)  # if we have prodcut_Id in data we will use that id  as (,id = record["ID"]) 
                                                            # bt we don't have so we will leave it, so elASTIC sEARECH WILL  assign and ID to each Document/record
    except Exception as e:
        print(e)

In [50]:
es.count(index="laptop_products")

# checking how many data are injested in index-elastic search ( we get 599 ie 600 documnet/record)


ObjectApiResponse({'count': 599, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## **Search Data**

In [66]:
input_key = "mac"
vector_of_input_key = model.encode(input_key)
query = {
    "field" : "DecriptionVector",
    "query_vector" : vector_of_input_key,
    "k" : 2,
    "num_candidates" : 600 , 
}

result = es.knn_search(index="laptop_products", knn=query, source=["link", "product_title", "price"])
# return the JSON - taking only  field oonly not all

result["hits"]["hits"]



C:\Users\acer\AppData\Local\Temp\ipykernel_9412\3544152129.py:10: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  result = es.knn_search(index="laptop_products", knn=query, source=["link", "product_title", "price"])


[{'_index': 'laptop_products',
  '_id': 'vaXiYJIBbAfGhzRatMN2',
  '_score': 0.45482057,
  '_source': {'link': 'https://www.daraz.com.np/products/apple-macbook-air-13-inch-m1-256gb-oliz-store-i106600811.html',
   'product_title': 'Apple MacBook Air 13-inch M1 256GB - Oliz Store',
   'price': 'Rs. 109,900'}},
 {'_index': 'laptop_products',
  '_id': '5aXiYJIBbAfGhzRaucOq',
  '_score': 0.45482057,
  '_source': {'link': 'https://www.daraz.com.np/products/apple-macbook-air-13-inch-m1-256gb-oliz-store-i106600811.html',
   'product_title': 'Apple MacBook Air 13-inch M1 256GB - Oliz Store',
   'price': 'Rs. 109,900'}}]